In [1]:
import os 
import music21
import music21.instrument
import numpy as np
import tensorflow as tf
import math
import time

2024-11-11 19:19:35.592361: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
with open('generated/melodyData.txt', 'r') as f:
    parts = eval(f.read())
    print ("Number of parts:" + str(len(parts)))

Number of parts:6410


In [3]:
# number of parts to train on
train_all_parts = True
num_data = int(1e9) if train_all_parts else 250
# max length of each part
max_sequence_length = int(1e9) if train_all_parts else 300 

# one-hot encoding
encodings = {}
encodingIndex = 0
for part in parts[:num_data]:
    for note in part[:max_sequence_length]:
        if note not in encodings:
            encodings[note] = encodingIndex
            encodingIndex += 1

# decoder constructed by reversing one-hot encoding
decodings = {}
for k, v in encodings.items():
    decodings[v] = k

# encode everything in a
data_encoded = []
for part in parts[:num_data]:
    data_encoded.append([encodings[note] for note in part[:max_sequence_length]])

num_data = min(num_data, len(data_encoded))


In [4]:
# one-hot encode the data
def generate_data(data_encoded, encodings):
    X = []
    Y = []
    # given data_encoded, generate training data by looping 
    for i in range(len(data_encoded)):
        currentX = []
        currentY = []
        for j in range(len(data_encoded[i])-1):
            currentX.append(data_encoded[i][j])
            currentY.append(data_encoded[i][j+1])
        X.append(currentX)
        Y.append(currentY)

    X_onehot = []
    for seq in X:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
        X_onehot.append(onehot)
    X = X_onehot
    
    Y_onehot = []
    for seq in Y:
        onehot = np.zeros((len(seq), len(encodings)))
        for note_index in range(len(seq)):
            onehot[note_index][seq[note_index]] = 1
            
        Y_onehot.append(onehot)
    Y = Y_onehot

    return X, Y

X_train, Y_train = generate_data(data_encoded[:math.floor(num_data*0.7)], encodings)
#X_test, Y_test = generate_data(data_encoded[10+math.floor(num_data*0.7)], encodings)
X_test, Y_test = generate_data(data_encoded[math.floor(num_data*0.7):], encodings)

# pads sequences so we can convert to numpy arrays
X_train = tf.keras.utils.pad_sequences(X_train, padding='pre')
Y_train = tf.keras.utils.pad_sequences(Y_train, padding='pre')

In [5]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Masking(mask_value=0, batch_input_shape=(1, None, len(encodings))),
    tf.keras.layers.LSTM(64, stateful=True, return_sequences=True),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(len(encodings), activation='softmax')
])

optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

model.summary()

if True:
    model.load_weights('checkpoints/model_full_v1.h5')
else:

    timeTaken = time.time()

    # Train the model one time step at a time
    accuracy = 0
    epochs = 2
    for epoch in range(epochs):
        print(f'Epoch {epoch + 1}/{epochs}')
        total_loss = 0
        for i, sequence in enumerate(X_train):
            # Reset states at the beginning of each sequence
            model.reset_states() 
            x = sequence.reshape((1, sequence.shape[0], len(encodings)))
            y = Y_train[i].reshape((1, sequence.shape[0], len(encodings)))
            loss, note_accuracy = model.train_on_batch(x, y)
            total_loss += loss
        accuracy += note_accuracy
        print(f"Accuracy {accuracy/(epoch+1)}, Loss {total_loss/len(X_train)}")
        print(f"Epoch time {time.time() - timeTaken}")
        timeTaken = time.time()
# 6.27 seconds for 5 epochs

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 masking (Masking)           (1, None, 188)            0         
                                                                 
 lstm (LSTM)                 (1, None, 64)             64768     
                                                                 
 dense (Dense)               (1, None, 64)             4160      
                                                                 
 dense_1 (Dense)             (1, None, 64)             4160      
                                                                 
 dense_2 (Dense)             (1, None, 188)            12220     
                                                                 
Total params: 85308 (333.23 KB)
Trainable params: 85308 (333.23 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [6]:
test_parts = 30
# tests model accuracy

correct = 0.0
total = 0


for i in range(min(test_parts, len(X_test))):
    # reset model
    model.reset_states()
    print (i)
    
    for j in range(len(X_test[i])):
        # use predict_on_batch
        
        curr_note = X_test[i][j].reshape(1, 1, X_test[i][j].shape[0])
        pred = model.predict_on_batch(curr_note)
        
        if (np.argmax(pred) == np.argmax(Y_test[i][j])):
            correct += 1
        total += 1
    # replace this, use train_on
print(correct/total)

0
1
2
3
4
5
6


KeyboardInterrupt: 

In [ ]:
# generate 

import numpy as np
import copy

def validNote(note):
    # ('minor', '4/4')
    if note[0] == 'minor' or note[0] == 'major':
        return False
    return True

def generate_melodies(num_melodies=3, measures=4, beats_per_measure=4):
    global model  # Use the global model variable
    beats_per_melody = measures * beats_per_measure
    forbidden_first_notes = set()
    
    original_states = copy.deepcopy(model.get_weights())

    generated_melodies = []

    melody_num = 0

    while melody_num < num_melodies:
  
        while True:
            one_hot = np.zeros(len(encodings))
            one_hot[1] = 1  

            pred = model.predict_on_batch(np.array([[one_hot]]))[0][0]

            if len(pred) != len(encodings):
                raise ValueError(f"Prediction size mismatch: {len(pred)} vs {len(encodings)}")

            first_note_index = np.random.choice(len(pred), p=pred)
            first_note = decodings[first_note_index]

            if first_note not in forbidden_first_notes and validNote(first_note) and first_note[0] != 'C':
                break

            model.set_weights(original_states)

        generated_notes = [first_note]
        previous_note_index = first_note_index

        beats = float(first_note[1])

        #for _ in range(1, beats_per_melody):
        while beats < beats_per_melody:
            one_hot = np.zeros(len(encodings))
            one_hot[previous_note_index] = 1
            
            pred = model.predict_on_batch(np.array([[one_hot]]))[0][0]

            if len(pred) != len(encodings):
                raise ValueError(f"Prediction size mismatch: {len(pred)} vs {len(encodings)}")

            next_index = np.random.choice(len(pred), p=pred)
            generated_note = decodings[next_index]
            generated_notes.append(generated_note)
            previous_note_index = next_index

            if generated_note[0] == 'C':
                beats += 1
            else:
                beats += float(generated_note[1])

        
        # TODO: if the length of notes is wrong or a single note is invalid, repeat the whole process and don't add it

        model.set_weights(original_states)

        # add it if it's valid
        if beats == beats_per_melody and all(validNote(note) for note in generated_notes):
            melody_output = []
            for measure in range(measures):
                measure_notes = generated_notes[measure * beats_per_melody: (measure + 1) * beats_per_melody]
                melody_output.extend(measure_notes)
            
            generated_melodies.append(melody_output)

            # add the first note to the forbidden list
            forbidden_first_notes.add(first_note)

            # if successful set this
            melody_num += 1
            print(f"Generating Melody {melody_num}:")
            print("-" * 30)
            print (generated_melodies[-1])

    return generated_melodies

melodies = generate_melodies(num_melodies=3)
#for melody in melodies:
#    for measure in melody:
#        print(measure)

Generating Melody 1:
------------------------------
[(8, '0.25'), (8, '0.25'), (12, '0.25'), (13, '0.25'), (8, '0.25'), (15, '0.25'), (8, '0.25'), (8, '0.25'), (12, '0.75'), (10, '0.25'), (10, '0.25'), (10, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (10, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (10, '0.25'), (15, '0.25'), (10, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (10, '0.25'), (14, '0.25'), (0, '0.25'), (10, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.50'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (15, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (15, '0.25'), (0, '0.25'), (15, '0.25'), (0, '0.25'), (15, '0.25'), (0, '0.25')]
Generating Melody 2:
------------------------------
[(15, '0.25'), (0, '0.25'), (10, '0.25'), (0, '0.25'), (11, '0.25'

In [54]:
from datetime import datetime
model.save_weights(f"./checkpoints/model_{datetime.now().strftime('%Y-%m-%d_%H-%M-%S')}.h5")